In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
from bench_util import *


emb_name = "cleanDeleteDynDict_150w"

word_dict,id_dict, matrix_normalized = load_embedding("D://gauntBaseline",emb_name)


dimensions 500
260377


In [2]:
#AutoDetect dims
#test with base2021_300
with open('.//'+ emb_name, 'r' , encoding="utf-8")  as file:
    line0 = file.readline()
    dimensions = len(line0.split())-1
print("dimensions "+str(dimensions))

dimensions 500


In [3]:
matrix = []
with open('.//'+ emb_name, 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        entry = line.split()
        word = entry[0].strip()
        values = entry[1:]
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values, "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

word_dict,id_dict,matrix_normalized = load

260377


In [6]:
def find_candidate(sim,exclusion_list_ids,isMult):
    for exclude_id in exclusion_list_ids:
        if isMult:
             sim[exclude_id] = 0#lowest possible value in 3CosMult
        else:
            sim[exclude_id] = -1#lowest possible value in 3CosAdd
    return np.argmax(sim)

In [5]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)# only use normalised version !!!
matrix = []

In [6]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list

def unzip(some_list):
    return [ i for i, j in some_list ]
    
# take nearest 6 and remove the searched word itself
#print(unzip(find_nearest_k('bank' ,7)[1:]))#the food or the place
#print(unzip(find_nearest_k('apple',7)[1:]))#location or to to speak to
#print(unzip(find_nearest_k('so'   ,7)[1:]))#noun (animal) or verb 
#print(unzip(find_nearest_k('desert',   6)[1:]))#animal or verb
#print(unzip(find_nearest_k('left',   6)[1:]))#adverb (direction) and verb(the plane left)
#print(unzip(find_nearest_k('duck',   6)[1:]))# The president of the bank walked along the river bank.
#print("--------")

# 3cosAdd normalised befor arithmetic

In [2]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim.numpy(),[id_a,id_a_star,id_b],False)
    return word_dict[index]

## 3cos Mult

In [3]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim.numpy(),[id_a,id_a_star,id_b],True)
    return word_dict[index]

# Test all

In [4]:
def format_results(add,mult):
    add_string = str(add)
    add_string = add_string.lstrip('0')
    add_string = format(add_string, ".4")
    mult_string = str(mult)
    mult_string = mult_string.lstrip('0')
    mult_string = format(mult_string, ".4")
    return add_string +'/'+mult_string


In [7]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0]
count_syn_sucess      = [0,0]

errors = []


files = os.listdir('D:\\datasets\\question-data')
for idx,file_name in enumerate(sorted(files)):
    print(file_name)
    if file_name.endswith('.classes'):
        continue
    
    with open('D:\\datasets\\question-data\\'+file_name,'r') as file:    
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
         
        for line_id,line in enumerate(lines):
            
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_sem_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_syn_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
        print(format_results(local_sucesses[0]/float(local_tasks_count),local_sucesses[1]/float(local_tasks_count)))
        print('-------------------')

print('semantical')
print(format_results(count_sem_sucess[0]/float(count_sem_questions),count_sem_sucess[1]/float(count_sem_questions)))

print('syntactical')
print(format_results(count_syn_sucess[0]/float(count_sem_questions),count_syn_sucess[1]/float(count_sem_questions)))

print('overall')
print(format_results((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_syn_questions), \
                     (count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_syn_questions)))

capital-common-countries.txt
.956/.956
-------------------
capital-world.txt


KeyboardInterrupt: 

In [10]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0]
count_syn_sucess      = [0,0]

errors = []


files = os.listdir('D:\\datasets\\question-data')
for idx,file_name in enumerate(sorted(files)):
    print(file_name)
    if file_name.endswith('.classes'):
        continue
    
    with open('D:\\datasets\\question-data\\'+file_name,'r') as file:    
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
         
        for line_id,line in enumerate(lines):
            
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_sem_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_syn_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
        print(format_results(local_sucesses[0]/float(local_tasks_count),local_sucesses[1]/float(local_tasks_count)))
        print('-------------------')

print('semantical')
print(format_results(count_sem_sucess[0]/float(count_sem_questions),count_sem_sucess[1]/float(count_sem_questions)))

print('syntactical')
print(format_results(count_syn_sucess[0]/float(count_sem_questions),count_syn_sucess[1]/float(count_sem_questions)))

print('overall')
print(format_results((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_syn_questions), \
                     (count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_syn_questions)))

capital-common-countries.txt
.956/.956
-------------------
capital-world.txt
.873/.851
-------------------
city-in-state.txt
.706/.705
-------------------
currency.txt
.019/.016
-------------------
family.txt
.583/.583
-------------------
gram1-adjective-to-adverb.txt
.159/.160
-------------------
gram2-opposite.txt
.110/.112
-------------------
gram3-comparative.txt
.286/.281
-------------------
gram4-superlative.txt
.079/.076
-------------------
gram5-present-participle.txt
.480/.476
-------------------
gram6-nationality-adjective.txt
.773/.772
-------------------
gram7-past-tense.txt
.483/.483
-------------------
gram8-plural.txt
.467/.461
-------------------
gram9-plural-verbs.txt
.308/.309
-------------------
ner.classes
treebank_pos.classes
universal_pos.classes
semantical
.731/.719
syntactical
.463/.460
overall
.542/.535


In [11]:
import winsound
winsound.Beep(440, 500)

Spearman Rank

In [12]:
with open(r'D:\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
58274
34268
tf.Tensor(0.9855161694508141, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
58274
58274
tf.Tensor(1.0, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
202953
35611
tf.Tensor(0.9874839451133623, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
156986
35611
tf.Tensor(0.9934372743184323, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
139248
176699
tf.Tensor(0.9950978196978985, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
118685
176699
tf.Tensor(0.9934937238796108, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
219262
74084
tf.Tensor(0.9788916549549911, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
64788
185448
tf.Tensor(0.9349484427835025, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
234074
107922
tf.Tensor(0.9847847419082645, shape=(), dtype=float64) 0.7
lineprofessor	doctor	6

linegovernor	interview	3.25

governor interview
34291
208478
tf.Tensor(0.9856263161887716, shape=(), dtype=float64) 0.325
linepractice	institution	3.19

practice institution
43450
246063
tf.Tensor(0.9885488809405643, shape=(), dtype=float64) 0.319
linecentury	nation	3.16

century nation
191459
159078
tf.Tensor(0.9918548542482013, shape=(), dtype=float64) 0.316
linecoast	forest	3.15

coast forest
163111
80216
tf.Tensor(0.9898517443494645, shape=(), dtype=float64) 0.315
lineshore	woodland	3.08

shore woodland
95473
188036
tf.Tensor(0.9742728411496613, shape=(), dtype=float64) 0.308
linedrink	car	3.04

drink car
245847
35611
tf.Tensor(0.9828903295300326, shape=(), dtype=float64) 0.304
linepresident	medal	3.00

president medal
139950
193411
tf.Tensor(0.9871194545196688, shape=(), dtype=float64) 0.3
lineprejudice	recognition	3.00

prejudice recognition
230329
192098
tf.Tensor(0.9702491448509636, shape=(), dtype=float64) 0.3
lineviewer	serial	2.97

viewer serial
244326
180445
tf.Tensor(0.962

In [13]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.12505254189485548, pvalue=0.07545200704465893)